In [1]:
import sys
from google.colab import drive
drive.mount('/content/gdrive') 
sys.path.append('/content/gdrive/MyDrive/My_env')
# ______________________________________________________________________

import os
import pandas as pd
import numpy as np
import pickle
from itertools import permutations, combinations
from tqdm.auto import tqdm
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import accuracy_score

from pycaret.utils import enable_colab
enable_colab()
from pycaret.classification import *

Mounted at /content/gdrive
Colab mode enabled.


In [ ]:
""" To Excute pycaret """
!pip uninstall sklearn -y
!pip install --upgrade sklearn
!pip install scikit-learn==0.23.2 --user

In [ ]:
dir = "/content/gdrive/MyDrive/ColabNotes/DACON/IncomeClassifier/"
data_dir = dir + "data/"
submission_file = "sample_submission.csv"
Y_train_file = "Y_train.csv"

submission = pd.read_csv(data_dir + submission_file)
Y_train = pd.read_csv(data_dir + Y_train_file)

In [ ]:
contest = "" 
dir = "/content/gdrive/MyDrive/ColabNotes/DACON/{}/".format(contest)
data_dir = dir + "data/"
submission_file = "sample_submission.csv"
test_file = "test_renew.csv"
X_train_file = "X_train.csv"
Y_train_file = "Y_train.csv"

submission = pd.read_csv(data_dir + submission_file)
test = pd.read_csv(data_dir + test_file)
X_train = pd.read_csv(data_dir + X_train_file)
Y_train = pd.read_csv(data_dir + Y_train_file)

In [ ]:
train_data = X_train.copy()
train_data['target'] = Y_train

train_data

## Execute AutoML with pycaret

In [ ]:
""" 학습 데이터 정보 """
exp_101 = setup(data = train_data, target = 'target', session_id = 113, fold_shuffle = True)

In [ ]:
""" 모델별 성능 """
best_model = compare_models()

## Modeling

In [ ]:
model = "" # e.g. "ada_pycaret", "rf_pycaret" etc. 
model_name = "" # e.g. "ada", "rf"

In [ ]:
""" 선택한 모델의 예측 성능 """
mod = create_model(model_name)

""" 하이퍼 파라미터 설정 """
tuned_mod = tune_model(mod)

""" 모델 확정 """
final_mod = finalize_model(tuned_mod)

In [ ]:
print("확정된 모델")
final_mod

In [ ]:
""" 확정된 모델로 학습 데이터 예측 """
train_pred = predict_model(final_mod, data = X_train)

""" 확정된 모델로 테스트 데이터 예측 """
final_pred = predict_model(final_mod, data = test)

In [ ]:
pred_train = train_pred["Label"]
pred_train_df = pd.DataFrame(pred_train)
pred_train_df.rename(columns = {"Label" : model}, inplace = True)
pred_train_df.head()

In [ ]:
pred_test = final_pred["Label"]
pred_test_df = pd.DataFrame(pred_test)
pred_test_df.rename(columns = {"Label" : model}, inplace = True)
pred_test_df.head()

In [ ]:
""" 학습 데이터 예측 결과 """
print(model, "- Accuracy : ", accuracy_score(Y_train, pred_train))

""" 테스트 데이터 예측 결과 """ 
print("\n* 테스트 데이터 예측 결과")
sns.countplot(x = pred_test_df[model])

In [ ]:
with open(dir + "pickle/pred_train_" + model + ".pickle", "wb") as ptr :
  pickle.dump(pred_train_df, ptr)
  
with open(dir + "pickle/pred_test_" + model + ".pickle", "wb") as pte :
  pickle.dump(pred_test_df, pte)